# IESB - Especialização em Inteligência Artificial

## Disciplina - Fundamentos de IA

### Alunos: David Guimarães Rocha / Walter Malta

# Contextualização

### Problema de classificação multiclasse (6 classes) para identificação de atividades realizadas por 30 pessoas em um estudo que captou dados do acelerômetro e do giroscópio de um celular Samsung Galaxy SII preso nas suas cinturas. Cada participante realizou 6 atividades diferentes (andar, subir escadas, descer escadas, sentar, ficar em pé e deitar) e obteve-se dados de aceleração e velocidade angular a partir dos 3 eixos de movimento em uma taxa constante de 50Hz. Os dados foram classificados manualmente pelos pesquisadores a partir de filmagens realizadas durante os estudos.

### Pré-processamento aplicado pelos pesquisadores: Os dados foram pré-processados aplicando-se um filtro de ruído e, em seguida, amostrados em janelas deslizantes de largura fixa de 2,56 seg e sobreposição de 50% (128 leituras / janela). O sinal de aceleração do sensor, que tem componentes gravitacionais e de movimento corporal, foi separado por meio de um filtro Butterworth na aceleração e gravidade do corpo. A força gravitacional é considerada como tendo apenas componentes de baixa frequência, portanto, um filtro com frequência de corte de 0,3 Hz foi usado. De cada janela, um vetor de características foi obtido pelo cálculo de variáveis ​​do domínio do tempo e da frequência (média, desvio padrão, MAD, etc).

### https://www.kaggle.com/uciml/human-activity-recognition-with-smartphones


### Features 

    Aceleração triaxial do acelerômetro (aceleração total) e a aceleração corporal estimada.

    Velocidade angular triaxial do giroscópio.

    Um vetor de 561 features com variáveis de domínio de tempo e frequência.

    A classe de atividade.


In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix as skcf
from sklearn.metrics import plot_roc_curve

# Organiza aleatoriamente matrizes 
from sklearn.utils import shuffle
from time import process_time

# validação cruzada
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
 


In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df = pd.read_csv('../input/activity-recognition-smartphone-merged-dataset/Human_Activity_Recognition_Using_Smartphones_Data.csv')
df.head()

In [ ]:
df.info()

In [ ]:
X = df.drop(columns='Activity')
y = df.Activity

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
scaler = StandardScaler().fit(X_train)

In [ ]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

A base contém um total de 10299 linhas e 562 colunas. A última coluna ("Activity") é a saída e tem 6 classes: 'LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS','WALKING_UPSTAIRS'.
Dividimos a base em treino e teste nas proporções de 80% e 20%.
A fração de treino será por sua vez utilizada na validação cruzada onde será subdividida novamente em treino e validação. 

### Base Model

Implementação com os parametros default de MLPClassifier 

```
hidden_layer_sizes=100, 
activation='relu', *, 
solver='adam', 
alpha=0.0001, 
batch_size='auto', 
learning_rate='constant', 
learning_rate_init=0.001, 
power_t=0.5, 
max_iter=200, 
shuffle=True, 
random_state=None, 
tol=0.0001, 
verbose=False, 
warm_start=False, 
momentum=0.9, 
nesterovs_momentum=True, 
early_stopping=False, 
validation_fraction=0.1, 
beta_1=0.9, 
beta_2=0.999, 
epsilon=1e-08, 
n_iter_no_change=10, 
max_fun=15000
```



In [ ]:
mlp_default = MLPClassifier()

In [ ]:
%%time
mlp_default.fit(X_train_scaled, y_train)

In [ ]:
y_pred_default = mlp_default.predict(X_test_scaled)
print(mlp_default.score(X_test_scaled, y_test))

In [ ]:
skcf(mlp_default, X_test_scaled, y_test, xticks_rotation='vertical', values_format = 'd')

In [ ]:
plt.plot(mlp_default.loss_curve_)
plt.show()

# Grafico LOSS

Implementação com 4 camadas

In [ ]:
mlp_base = MLPClassifier(hidden_layer_sizes=(256,128,64,32),activation="relu",random_state=1) # implementação de MLP que encontramos no Kaggle.

In [ ]:
%%time
mlp_base.fit(X_train_scaled, y_train)

In [ ]:
y_pred_base = mlp_base.predict(X_test_scaled)
print(mlp_base.score(X_test_scaled, y_test))

In [ ]:
skcf(mlp_base, X_test_scaled, y_test, xticks_rotation='vertical', values_format = 'd')

In [ ]:
plt.plot(mlp_base.loss_curve_)
plt.show()

# Grafico LOSS

### Validação cruzada

In [ ]:
'''
%%time

# gridsearchCV

param_grid = {'hidden_layer_sizes': [(100,), (200,)],
              'max_iter': [100, 200], 
              'activation': ['logistic', 'tanh', 'relu'], 
              'solver': ['sgd', 'adam'], 
              'alpha': [0.001, 0.0001],
              'learning_rate_init': [0.001, 0.0001],
              'momentum': [0.5, 0.1] 
              }

grid_mlp = GridSearchCV(estimator=MLPClassifier(early_stopping=True, random_state=1), param_grid=param_grid, scoring='roc_auc_ovo_weighted', cv=3, n_jobs=-1)

grid_mlp.fit(X_train_scaled, y_train)

print(grid_mlp.best_estimator_, grid_mlp.best_score_, grid_mlp.best_params_)


MLPClassifier(alpha=0.5, early_stopping=True, hidden_layer_sizes=(300,),
              learning_rate_init=0.01, momentum=0.5, random_state=1,
              solver='lbfgs') 0.9882719420690285 {'activation': 'relu', 'alpha': 0.5, 'hidden_layer_sizes': (300,), 'learning_rate_init': 0.01, 'momentum': 0.5, 'solver': 'lbfgs'}
Wall time: 1h 11min 43s
'''

In [ ]:
# Função para calculo da precisão em cada classe de saida
# matriz = matriz confusão,  lista = lista de classes 
# resultado:  soma = vetor com o percentual de precisão das classes

def precisao(matriz,lista):
    soma = []
    total = matriz.sum(axis=0)
    for i in range(len(lista)):
        soma.append(100 * matriz[i,i] / total[i])
    return soma

In [ ]:
# Você deve consultar a documentação de cada classificador para ter uma noção geral de seus principais parâmetros.
# classificadores

Nclass = 2
epocas = 1000
#epocas = 10
k = 10 # para validação cruzada

In [ ]:
#com o k folds definidos, separa dados de validação e treinamento - repetir k vezes (folds)

time_train_mlp =[]
#acuracia_mlp = []
precisao_mlp = []
#score_mlp = []
acuracia_mlp = []

In [ ]:
mlp_best = MLPClassifier(hidden_layer_sizes=(200,), 
                         activation='tanh', 
                         max_iter=epocas,
                         solver='adam', 
                         learning_rate_init=0.001,
                         alpha=0.001, 
                         # momentum=0.5, não utilizado pois o solver é o adam
                         random_state=1)

# mlp_best.get_params()

# 'activation': 'tanh',  'alpha': 0.001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (200,), 'learning_rate': 'constant',
# 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 1000, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 1, 'shuffle': True,
# 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False

In [ ]:
# Teste com parametros usados em sala de aula 
#mlp_best = MLPClassifier(hidden_layer_sizes=(256,), activation='relu', max_iter=epocas, alpha=1e-4,
#                    solver='sgd', verbose=10, tol=1e-10, random_state=1, learning_rate_init=.01)
# Precisão para LAYING: 99.87%
# Precisão para SITTING: 94.02%
# Precisão para STANDING: 93.83%
# Precisão para WALKING: 99.28%
# Precisão para WALKING_DOWNSTAIRS: 98.56%
# Precisão para WALKING_UPSTAIRS: 99.12%

In [ ]:
# Validação cruzada com k folds
skf = StratifiedKFold(n_splits=k, random_state=None)

In [ ]:
#classification_reports metrics

In [ ]:
y_train_array = y_train.to_numpy()
y_train_array

In [ ]:
%%time
for train_index, val_index in skf.split(X_train_scaled, y_train_array): 

    print("Train:", train_index, "Validation:", val_index) 
    
    x_treinamento, x_validacao = X_train_scaled[train_index], X_train_scaled[val_index] 
    y_treinamento, y_validacao = y_train_array[train_index], y_train_array[val_index]
    
    # para garantir serem randomicos
    x_treinamento, y_treinamento = shuffle(x_treinamento, y_treinamento, random_state = 42)
    x_validacao, y_validacao = shuffle(x_validacao, y_validacao, random_state = 42)

    # Aqui fazer um treinamento de x épocas e uma validacao da MLP
    start = process_time()
    mlp_best.fit(x_treinamento, y_treinamento) # são 10 treinamentos - no final faz a média
    end = process_time()
    time_mlp = end - start

    # Métricas da validacão mlp
    preds_val_mlp = mlp_best.predict(x_validacao)  # E 10 validações - no final faz a média
    cm_val_mlp = confusion_matrix(y_validacao, preds_val_mlp)
    acuracia_mlp_ = accuracy_score(y_validacao, preds_val_mlp, normalize=False)

    print(f'Acurácia: {100*acuracia_mlp_/y_validacao.shape[0]:.2f}%')
    print(f'Nr linhas de validação: {y_validacao.shape[0]}')
    print(f'Nr de linhas de predicao: {preds_val_mlp.shape[0]}')  
    

    precisao_mlp_ = precisao(cm_val_mlp,mlp_best.classes_)

    
    time_train_mlp.append(time_mlp)
    acuracia_mlp.append(100*acuracia_mlp_/y_validacao.shape[0])
    precisao_mlp.append(precisao_mlp_)

In [ ]:
media_time_train_mlp = sum(time_train_mlp) / float(len(time_train_mlp))
media_acuracia_mlp = sum(acuracia_mlp) / float(len(acuracia_mlp))


print('Tempo médio de treinamento MLP com ' + str(k) + ' kfold ' + str (media_time_train_mlp))
print('Médias das Validações com ' + str(k) + ' folds')
print('Acurácia_mlp: ' + str(media_acuracia_mlp))

In [ ]:
#Calculo da precisão para as classes de saida
soma_colunas_precisao_mlp = np.sum(precisao_mlp,axis=0)
numero_de_linhas_de_precisao_mlp = len(precisao_mlp)
media_de_precisao_mlp = soma_colunas_precisao_mlp/numero_de_linhas_de_precisao_mlp
rotulos_matriz_confusao = mlp_best.classes_
indice_rotulo_laying = np.where(mlp_best.classes_ == 'LAYING')[0][0]
indice_rotulo_sitting = np.where(mlp_best.classes_ == 'SITTING')[0][0]
indice_rotulo_standing = np.where(mlp_best.classes_ == 'STANDING')[0][0]
indice_rotulo_walking = np.where(mlp_best.classes_ == 'WALKING')[0][0]
indice_rotulo_walking_downstairs = np.where(mlp_best.classes_ == 'WALKING_DOWNSTAIRS')[0][0]
indice_rotulo_walking_upstairs = np.where(mlp_best.classes_ == 'WALKING_UPSTAIRS')[0][0]
media_precisao_mlp_laying = media_de_precisao_mlp[indice_rotulo_laying]
media_precisao_mlp_sitting = media_de_precisao_mlp[indice_rotulo_sitting]
media_precisao_mlp_standing = media_de_precisao_mlp[indice_rotulo_standing]
media_precisao_mlp_walking = media_de_precisao_mlp[indice_rotulo_walking]
media_precisao_mlp_walking_upstairs = media_de_precisao_mlp[indice_rotulo_walking_upstairs]
media_precisao_mlp_walking_downstairs = media_de_precisao_mlp[indice_rotulo_walking_downstairs]
print(f'Precisão para LAYING: {media_precisao_mlp_laying:.2f}%')
print(f'Precisão para SITTING: {media_precisao_mlp_sitting:.2f}%')
print(f'Precisão para STANDING: {media_precisao_mlp_standing:.2f}%')
print(f'Precisão para WALKING: {media_precisao_mlp_walking:.2f}%')
print(f'Precisão para WALKING_DOWNSTAIRS: {media_precisao_mlp_walking_upstairs:.2f}%')
print(f'Precisão para WALKING_UPSTAIRS: {media_precisao_mlp_walking_downstairs:.2f}%')

In [ ]:
y_pred = mlp_best.predict(X_test_scaled)

# score da classificação realizada no dataset de teste com o melhor modelo encontrado com GridSearchCV e validação cruzada

print(mlp_best.score(X_test_scaled, y_test))

In [ ]:
# matriz de confusão do melhor modelo

skcf(mlp_best, X_test_scaled, y_test, xticks_rotation='vertical', values_format = 'd')

In [ ]:
plt.plot(mlp_best.loss_curve_)
plt.show()

# Grafico LOSS

## Conclusões

As principais dificuldades encontradas estão relacionadas à compreensão do algorítmo, uma vez que os dados foram bem pré-processados, estruturados e novas features já tinham sido geradas pelos pesquisadores que realizaram a pesquisa (aplicamos apenas um Scaler). Assim, buscamos entender o funcionamento do algorítmo e seus parâmetros, afim de melhorar a performance do modelo.

De início, é interessante identificar a função de ativação e o "solver" (que faz a otimização dos pesos) que mais se adequam ao problema em questão, quantidade de dados e as features disponíveis. 

A principal conclusão que chegamos é em relação à quantidade de camadas internas e neurônios. O aumento infundado no número de camadas e neurônios não nos parece aumentar a performance do modelo por si só (e ainda aumentar o tempo de processamento), sem estar combinada com o ajuste e otimização de outros parâmentros, como velocidade de aprendizagem, regularização, quantidade de iterações e momentum.

Ao utilizar o classificador perceptron multicamadas com os valores 'default' (1 camada, 100 neurônios) obtivemos a performance mais baixa, mas ainda assim bastante elevada, de 98,83%.
Com a rotina GridSearchCV testamos diversos modelos e encontramos um com apenas 1 camada interna e 200 neurônios, com ajuste de outros parâmetros, que performou basicamente igual um modelo mais complexo, com quatro camadas internas e valores de parâmetros default. Entretanto, o modelo mais simples, com apenas 1 camada interna, foi treinado quase 2x mais rápido. 

Efetuamos a validação cruzada e a matriz confusão apresentou neste caso pequenas variações, ainda assim a acurácia do modelo permaneceu a mesma encontrada incialmente: 99,08%.
Observamos que a precisão encontrada em todas as classes de saída foi bastante elevada sendo para 'SITTING' de 96.37%, a menor, e para 'LAYING' de 99.81%, a mais alta. 

Plotamos a função de perda ("loss function") para as diferentes configurações e percebemos que o formato da curva foi bem semelhante em todos os casos, apresentando uma rápida queda inicialmente, indicando uma rápida convergência e depois se estabilizando e se aproximando de uma reta constante. O modelo com 4 camadas apresentou no grafico da função de perda uma oscilação maior, sendo mais um indicador que modelos mais simples com apenas uma camada interna são mais indicados para a base de dados analisada.

Existem diversos trabalhos publicados no próprio Kaggle que fazem a implementação de vários algorítmos para fazer essa classificação, incluindo um algoritmo que faz o "stacking" de diversos classificadores (https://www.kaggle.com/kabirnagpal/feature-selection-and-stacking-f1-score-99).

Nos parece não ser necessária uma rede neural complexa para resolver esse problema, uma vez que temos um bom pré-processamento. Como sugestão, e a depender da aplicação, pode-se avaliar a escolha de um algoritmo menos complexo, que tenha um pequeno comprometimento da performance mas que ganhe em termos de processamento.

In [ ]:
#!pip install category_encoders